# End-to-End FastAPI Pipeline: Code → Docker → Kubernetes

Hi, I’m walking you through a full pipeline:
1. **Python API**: health check, GET and POST endpoints
2. **Containerization**: Dockerfile, build, run
3. **Docker Hub**: create repo, tag, push
4. **Kubernetes**: enable, deploy YAML, inspect

All code uses dummy data so you can follow along.

## 1. Define our FastAPI app (`server.py`)
We'll create three endpoints:
- `GET /` for a health check
- `GET /user/{userid}` returning dummy user data
- `POST /user` accepting a JSON body


In [ ]:
%%bash
cat > server.py << 'EOF'
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI()

# Dummy in-memory storage
library = {
    111: {"name": "Alice", "date": "2025-04-25", "service": "DemoService"}
}

class UserIn(BaseModel):
    name: str
    date: str
    service: str

@app.get("/", tags=["health"])
async def healthcheck():
    return {"status": "healthy"}

@app.get("/user/{userid}", tags=["user"])
async def get_user(userid: int):
    user = library.get(userid)
    if not user:
        raise HTTPException(status_code=404, detail="User not found")
    return user

@app.post("/user", tags=["user"])
async def create_user(user: UserIn):
    new_id = max(library.keys()) + 1
    library[new_id] = user.dict()
    return {"id": new_id, **user.dict()}
EOF

echo "Created server.py:" && sed -n '1,20p' server.py

## 2. `requirements.txt`
List our dependencies.

In [ ]:
%%bash
cat > requirements.txt << 'EOF'
fastapi
uvicorn[standard]
EOF

echo "requirements.txt:" && cat requirements.txt

## 3. Build & Run Locally with Docker
**Dockerfile**:

In [ ]:
%%bash
cat > Dockerfile << 'EOF'
FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY server.py .
CMD ["uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8000"]
EOF

echo "Dockerfile:" && sed -n '1,20p' Dockerfile

### Build and run:

In [ ]:
%%bash
# Build the Docker image
docker build -t fastapi-demo:latest .

# Run the container
docker run --rm -d -p 8000:8000 --name fastapi-demo fastapi-demo:latest

echo "Container running at http://localhost:8000"


## 4. Push to Docker Hub
1. Create a repo on Docker Hub (e.g. `yourdockerid/fastapi-demo`).

2. Tag & push:

In [ ]:
%%bash
# Tag the image
docker tag fastapi-demo:latest yourdockerid/fastapi-demo:latest

# Push to Docker Hub
docker push yourdockerid/fastapi-demo:latest


## 5. Enable Kubernetes (Docker Desktop or Minikube)
- **Docker Desktop**: Settings → Kubernetes → Enable → Apply & Restart
- **Minikube**: `minikube start`

## 6. Kubernetes Manifests
### Deployment (`deployment.yaml`)


In [ ]:
%%bash
cat > deployment.yaml << 'EOF'
apiVersion: apps/v1
kind: Deployment
metadata:
  name: fastapi-demo
spec:
  replicas: 3
  selector:
    matchLabels:
      app: fastapi-demo
  template:
    metadata:
      labels:
        app: fastapi-demo
    spec:
      containers:
      - name: fastapi-demo
        image: yourdockerid/fastapi-demo:latest
        ports:
        - containerPort: 8000
EOF

echo "deployment.yaml:" && sed -n '1,20p' deployment.yaml

### Service (`service.yaml`)


In [ ]:
%%bash
cat > service.yaml << 'EOF'
apiVersion: v1
kind: Service
metadata:
  name: fastapi-demo-svc
spec:
  selector:
    app: fastapi-demo
  type: LoadBalancer
  ports:
  - port: 80
    targetPort: 8000
EOF

echo "service.yaml:" && sed -n '1,20p' service.yaml

## 7. Deploy to Kubernetes


In [ ]:
%%bash
kubectl apply -f deployment.yaml
kubectl apply -f service.yaml
kubectl get pods,svc


## 8. Verify & Inspect
- **Check pods**: `kubectl get pods -l app=fastapi-demo`
- **Logs**: `kubectl logs <pod-name>`
- **Service endpoint**: e.g. `http://localhost/user/111` or `curl http://localhost/user`

---
This notebook walked through:
1. Writing a FastAPI server with health, GET, POST endpoints
2. Packaging into Docker
3. Pushing to Docker Hub
4. Enabling Kubernetes
5. Creating Deployment & Service YAMLs
6. Deploying & inspecting the running app

Feel free to customize and share this end-to-end example!